In [ ]:
# Bigger dataset available here http://s3.amazonaws.com/hr-testcases-us-east-1/329/assets/bigger_data_set.txt

The first line contains three space separated numbers m k d.
* m - the amount of money you could spend that day.
* k - the number of different stocks available for buying or selling.
* d - the number of remaining days for trading stocks.

k lines follow, each in the following format: name owned prices
* name - the name of the stock (a string).
* owned - the number of shares you own of that stock.
* prices - 5 space separated numbers representing the stock's price for the last 5 days. These are ordered from oldest to newest, so the last number is the current stock price.

Sample input

100 4 400

AA 10 10.01 15.01 20.01 25.01 30.01

BB 5 10.01 20.01 30.01 40.01 50.01

CC 6 10.01 9.01 8.01 7.01 6.01

DD 7 10.01 8.01 6.01 4.01 2.01

In [1]:
import pandas as pd

In [2]:
# Take first line of input, it contains useful info.
info = input().split()
m = float(info[0])  # m - the amount of money you could spend that day.
k = int(info[1])  # k - the number of different stocks available for buying or selling.
d = int(info[2])  # d - the number of remaining days for trading stocks.

100 4 400


In [3]:
# Take input info about the stocks .
inpt = [input().split()for line in range(k)]

AA 10 10.01 15.01 20.01 25.01 30.01
BB 5 10.01 20.01 30.01 40.01 50.01
CC 6 10.01 9.01 8.01 7.01 6.01
DD 7 10.01 8.01 6.01 4.01 2.01


In [4]:
# Formatting updates.
for i in range(k):
    for j in range(2,7):
        inpt[i][j] = float(inpt[i][j])   
    inpt[i][1] = int(inpt[i][1])

        
inpt

[['AA', 10, 10.01, 15.01, 20.01, 25.01, 30.01],
 ['BB', 5, 10.01, 20.01, 30.01, 40.01, 50.01],
 ['CC', 6, 10.01, 9.01, 8.01, 7.01, 6.01],
 ['DD', 7, 10.01, 8.01, 6.01, 4.01, 2.01]]

In [5]:
def Slope_calc(lst):
    import numpy as np
    
    name = lst[0]
    owned = lst[1]
    prices = lst[2:]
    
    # create sequential array representing days
    days = np.arange(5)
    # Use numpy method polyfit() with degree '1' to get the slope value
    fit = np.polyfit(days, prices, 1)
    
    
    return([name, owned, prices[-1], fit[0]])

In [6]:
# Creating dataframe for decision making                  

tmp_lst = []
for i in range(len(inpt)):
    tmp_lst.append(Slope_calc(inpt[i]))

df = pd.DataFrame(tmp_lst, columns=['name', 'owned', 'today price', 'slope'])

# Adding ACTION and QTY columns:
df['ACTION'] = ''
df['QTY'] = 0

# Sort by descending slope meaning the more valueable stock to buy.
df = df.sort_values(by=['slope'], ascending = False)

In [7]:
df

,name,owned,today price,slope,ACTION,QTY
1,BB,5,50.01,10.0,,0
0,AA,10,30.01,5.0,,0
2,CC,6,6.01,-1.0,,0
3,DD,7,2.01,-2.0,,0


In [8]:
# This is attempt to control greediness of the algorithm spending all money on stock with the highest slope.
# Calculate positive slope weights as ratio to sum of all positive slope values.

for index, row in df.iterrows():
    money_coeff = m
    if row['slope'] > 0:
        df.at[index,'ACTION'] = 'BUY'
        df.at[index,'QTY'] = ( money_coeff*row['slope']/sum(df['slope'][df.slope>0]) ) // row['today price']
        m -= row['today price'] * row['QTY']
        
    if row['slope'] < 0:
            if row['owned'] > 0:
                df.at[index,'ACTION'] = 'SELL'
                df.at[index,'QTY'] = int(row['owned'])
    
    if m>0 and row['slope'] > 0:
        df.at[index,'QTY'] = row['QTY'] + m//row['today price']
        m -= row['QTY'] * row['today price']

df

,name,owned,today price,slope,ACTION,QTY
1,BB,5,50.01,10.0,BUY,1
0,AA,10,30.01,5.0,BUY,3
2,CC,6,6.01,-1.0,SELL,6
3,DD,7,2.01,-2.0,SELL,7


In [9]:
# Filter rows where the action was taken (meaning QTY is more than zero)
df_actionable = df[df.QTY>0]

# Print transaction
print(len(df_actionable))
for index, row in df_actionable.iterrows():
    print(row['name'], row['ACTION'], row['QTY'])

4
BB BUY 1
AA BUY 3
CC SELL 6
DD SELL 7


# HackerRank upload

In [ ]:
import pandas as pd

# Take first line of input, it contains useful info.
info = input().split()
m = float(info[0])  # m - the amount of money you could spend that day.
k = int(info[1])  # k - the number of different stocks available for buying or selling.
d = int(info[2])  # d - the number of remaining days for trading stocks.

# Take input info about the stocks .
inpt = [input().split()for line in range(k)]

# Formatting updates.
for i in range(k):
    for j in range(2,7):
        inpt[i][j] = float(inpt[i][j])    
    inpt[i][1] = int(inpt[i][1])

        

def Slope_calc(lst):
    import numpy as np
    
    name = lst[0]
    owned = lst[1]
    prices = lst[2:]
    
    # create sequential array representing days
    days = np.arange(5)
    # Use numpy method polyfit() with degree '1' to get the slope value
    fit = np.polyfit(days, prices, 1)
    
    
    return([name, owned, prices[-1], fit[0]])



# Creating dataframe for decision making                  

tmp_lst = []
for i in range(len(inpt)):
    tmp_lst.append(Slope_calc(inpt[i]))

df = pd.DataFrame(tmp_lst, columns=['name', 'owned', 'today price', 'slope'])

# Adding ACTION and QTY columns:
df['ACTION'] = ''
df['QTY'] = 0

# Sort by descending slope meaning the more valueable stock to buy.
df = df.sort_values(by=['slope'], ascending = False)



# This is attempt to control greediness of the algorithm spending all money on stock with the highest slope.
# Calculate positive slope weights as ratio to sum of all positive slope values.

if d > 1:
    for index, row in df.iterrows():
        money_coeff = m
        if row['slope'] > 0:
            df.at[index,'ACTION'] = 'BUY'
            df.at[index,'QTY'] = ( money_coeff*row['slope']/sum(df['slope'][df.slope>0]) ) // row['today price']
            m -= row['today price'] * row['QTY']

        if row['slope'] < 0:
                if row['owned'] > 0:
                    df.at[index,'ACTION'] = 'SELL'
                    df.at[index,'QTY'] = int(row['owned'])

        if m>0 and row['slope'] > 0:
            df.at[index,'QTY'] = row['QTY'] + m//row['today price']
            m -= row['QTY'] * row['today price']

else:
    for index, row in df.iterrows():
        if row['owned'] > 0:
            df.at[index,'ACTION'] = 'SELL'
            df.at[index,'QTY'] = int(row['owned'])


# Filter rows where the action was taken (meaning QTY is more than zero)
df_actionable = df[df.QTY>0]


# Print transaction
print(len(df_actionable))
for index, row in df_actionable.iterrows():
    print(row['name'], row['ACTION'], row['QTY'])

In [ ]:
df

In [ ]:
100 10 20
CAL 0 121.83 122.26 123.94 122.11 120.58
UCB 0 52.34 50.89 47.62 51.16 52.4
RIT 0 100.21 99.33 102.87 110.63 110.72
UCLA 0 12.5 27.22 9.32 16.07 3.86
USC 0 249.58 244.24 242.56 245.13 245.35
UFL 0 20.21 19.77 21.34 20.21 21.17
UMAD 0 109.39 120.49 131.35 127.97 121.38
RICE 0 126.3 128 129.08 129.28 124.44
UMD 0 102.96 103.62 98.22 96.6 99.1
UCSC 0 213.77 193.51 178.53 180.08 208.29